# Homework 8

## CSCI E-82A


In the a previous homework assignments, you used two different dynamic programming algorithm to solve a robot navigation problem by finding optimal paths to a goal in a simplified warehouse environment. Now you will use classical reinforcement learning algorithms to find optimal paths in the same environment.

>**Note:** This assignment involves modifying code from several lesson notebooks. Please ask any questions about code on the course Piazza forum. 

Execute the code in the cell below to load the packages you will need for this exercise. 

In [ ]:
## import required packages
import numpy as np
import numpy.random as nr
import itertools
import pandas as pd
from copy import deepcopy

The configuration of the warehouse environment is illustrated in the figure below.

<img src="GridWorldFactory.JPG" alt="Drawing" style="width:200px; height:200px"/>
<center> **Grid World for Factory Navigation Example** </center>

The goal is for the robot to deliver some material to position (state) 12, shown in blue. Since there is a goal state or **terminal state** this an **episodic task**. 

There are some barriers comprised of the states $\{ 6, 7, 8 \}$ and $\{ 16, 17, 18 \}$, shown with hash marks. In a real warehouse, these positions might be occupied by shelving or equipment. We do not want the robot to hit these barriers. Thus, we say that transitioning to these barrier states is **taboo**.

As before, we do not want the robot to hit the edges of the grid world, which represent the outer walls of the warehouse. 

## Representation

You are, no doubt, familiar with the representation for this problem by now.    

As with many such problems, the starting place is creating the **representation**. In the cell below encode your representation for the possible action-state transitions. From each state there are 4 possible actions:
- up, u
- down, d,
- left, l
- right, r

There are a few special cases you need to consider:
- Any action transitioning state off the grid or into a barrier (taboo) state should keep the state unchanged. 
- Any action in the goal state keeps the state unchanged. 
- Any transition within the taboo (barrier) states can keep the state unchanged. If you experiment, you will see that other encodings work as well since the value of a barrier states are always zero and there are no actions transitioning into these states. 

In the cell below code the **dictionary  of dictionaries** which defines the possible state transitions to the neighboring states.

> **Hint:** It may help you create a pencil and paper sketch of the transitions, rewards, and probabilities or policy. This can help you to keep the bookkeeping correct. 

In [ ]:
## Define the transition dictonary of dictionaries:


The robot receives the following rewards:
- +10 for entering position 12. 
- -1 for attempting to leave the grid. In other words, we penalize the robot for hitting the edges of the grid.  
- -0.1 for all other state transitions, which is the cost for the robot to move from one state to another. If we did not have this penalty, the robot could follow any random plan to the goal which did not hit the edges. 

This **reward structure is unknown to the MC RL agent**. The agent must **learn** the rewards by sampling the environment. 

In the code cell below encode the **dictionary  of dictionaries** for your representation of this reward structure you will use in your simulated environment.  

### Environment Simulator

You must create a simulator for the grid world environment in the code below. Your RL agents will have no information on the environment, and will only interact through the simulator. The simulator must have at least the following characteristics:

1. Use the representations of the environment defined above.
2. Have arguments of state, action, terminal state and environment representation. 
3. Returns the state-prime, reward and if the state is the terminal state. 

You will also need a utility function to determine if a state is either terminal or taboo. This function will be used for generating random starts. 

### Start Episode Function

To do random starts, you will need a function to generate them. Your function must ensure that the start state is not the terminal state or a taboo state. In the cell below create and test the required code.  

### Initial Uniform Policy 

You need to define the initial transition probabilities for the Markov process. In the cell below create a **dictionary  of dictionaries** to set the probabilities for each transition as a **uniform distribution** leading to random action by the robot. 

> **Note:** As these are just starting values, the exact values of the transition probabilities are not actually all that important in terms of solving the RL problem. Also, notice that it does not matter how the taboo state transitions are encoded. The point of the RL algorithm is to learn the transition policy.

You will find it useful to create a list of taboo states, which you can encode in the cell below.

## Monte Carlo State Value Policy Evaluation

With your representations and the environment simulator defined, you can now create and test functions to perform MC RL **policy evaluation**. 

In the cell below create and test a function for taking an action in the environment. You may start with the `take_action` function from the lesson notebook. This function should have the following characteristics:

1. Have arguments of state and policy.
2. Compute the action, given the probabilities of the policy.
2. Return the action, the successor state, the reward, and a flag indicating if the successor state is terminal.  

### Monte Carlo Episode Generation

As a first step you will need a function to generate a single first visit Monte Carlo episode for state value estimation, given the policy in the warehouse environment. You may start with the `MC_generate_episode` function from the MC RL notebook as a starting point.    

Make sure that taboo states are not visited on the random walk and that each episode ends at the terminal state. These key properties are 


Why is the algorithm used in the `MC_generate_episode` considered first visit Monte Carlo?

ANS: 

Now you have all the components in place to use the Monte Carlo algorithm to compute state values. You may use the `MC_state_values` function from the lesson notebook as a starting point. The main work done in this function is to iterate over the required number of episodes to compute the MC state value estimates. 

Execute your function for 5,000 episodes, to evaluate the initial uniform policy. 

Make sure that the state values of the terminal state and taboo state must be zero. If they are not, there must be something wrong with your code.  

Examine your results. Are do the state values decrease with distance from the terminal state, and is this consistent with sampling the gain with the Monte Carlo algorithm? With 5,000 episodes sampled can you see evidence of the high variance inherent in MC methods? To answer the this second question it might help you to re-compute the state values using fewer or more episodes. 

ANS 1:      
ANS 2:    

## Monte Carlo Policy Improvement

Now you are ready to perform Monte Carlo policy improvement. As a first step you will need a function to compute the action values (Q values) for one episode. You may start with the `MC_action_value_episode` function from the lesson notebook. This function takes the following arguments, the current policy dictionary, the Q numpy array, the number of visits numpy array, the starting state, the number of states, and the number of actions. The function returns the updated Q numpy array and the number of visits numpy array. In the cell below create and test this function.     

In [ ]:
import itertools


Next, in the cell below you need to create and test a function to compute the action value estimates over a number of episodes. You may start with the 'MC_action_values' function from the TD lesson notebook. The function has arguments of the initial policy, the action value matrix, the number of episodes to sample and the inital or starting state. The function returns the action value matrix. Execute your function for 10000 episodes, saving the results in a numpy array, and print the array.  

In [ ]:
def print_Q(Q):
    Q = pd.DataFrame(Q, columns = ['up', 'down', 'left', 'right'])
    print(Q)



Make sure that the taboo states and the terminal state all have zero action values for every action. If not, there is a bug in your code. 

Find the largest action values in this array. Are these values consistent with the reward structure of this problems and why?

ANS:  

With the action value array computed, in the cell below you can create and test code to find an improved policy. You may use the `update_policy` function from the TD lesson notebook. The function has arguments of an initial policy, an action value array, the $\epsilon$-greedy parameter, and returns an improved policy. 

Before executing your `update_policy` function will need to create a deep copy of your initial dictionary. Do this with the [copy.deepcopy function](https://docs.python.org/3/library/copy.html).  

Execute your code with $\epsilon = 0.01$, and print the policy computed.

Examine your impoved policy. Are the most probable actions consitent with an optimal policy and why? 

ANS: 

Now, compute and print the Monte Carlo state values for the improved policy in the cell below. Use 10,000 episodes to compute these state value estimates. 

Compare these state values to the MC state values you computed using the initial random policy. Is there a clear improvement across all states?    

ANS:   

## TD(0) Policy Evaluation

With the analysis using Monte Carlo RL algorithms completed, you will perform the same analysis using the TD(0)/SARSA(0) methods. You will then be able to compare the results using these two methods.  

As a first step you will need a function to compute state values using the TD(0) algorithm. You are welcome to start with the `td_0_state_values` function from the TD learning notebook. The arguments to this function are policy the number of episodes, alpha (the learning rate) and gamma (discount factor). This function uses the `take_action` function you defined previously. Keep in mind that you may need to modify this code to correctly treat the taboo states. Specifically, taboo states should not be visited. 

Execute your code for the initial policy to test it using 20,000 episodes, a learning rate of 0.01 and a discount rate of 1.0.  

Make sure that the state values of the terminal state and taboo state must be zero. If they are not, there must be something wrong with your code.  

Examine your results. Do these state values seem consistent with what you would expect for this problem, and why? Compare these results to the unbiased results of the MC state value estimates. Can you see evidence of the bias of the TD(0) algorithm, and if so, what is it? To explore the answer to the second question you can try different values of the learning rate, alpha and see how this affects the bias. If you use small values of alpha, you will need to increase the number of episodes sampled.

ANS 1:    
ANS 2:   


## SARSA(0) Policy Improvement

Now you will perform policy improvement using the SARSA(0) algorithm.  You are welcome to start with the `new_episode` and `SARSA_0` functions from the TD/Q-learning notebooks.    

The `new_episode` function starts an episode to sample action values. This function has arguments of number of states and the policy. The function calls the `start_episode` and `take_action` functions you created previously. The function returns the starting state of the episode, the action, the successor state, the reward, and a flag indicating if the successor state is terminal. 

The `SARSA_0` function estimates action value (Q) over a specified number of episodes. The function has arguments of policy, number of episodes sampled, the learning rate (alpha) and the discount rate (gamma). The function uses the `new episode` and `take_action` functions. The state and successor state information is updated at each time step. An action value array is returned.  

Execute your code for 20,000 episodes, and with $\alpha = 0.05$, and $\gamma = 1.0$.

Examine the action values you have computed. Ensure that the action values are 0 for the goal and taboo states. Also check that the actions with the largest values for each state make sense in terms of reaching the goal. 

With the action values computed, you will now find an improved policy. Make sure you create a copy of the initial random policy with `copy.deepcopy`. Execute the `update_policy` function using the copy of the initial random policy, the action value array you computed, and with $\epsilon = 0.01$.

Examine the policy you have computed. Make sure that the most probable actions lead to an optimal path to the terminal state from any other state, that is not taboo. 

Now, you will evaluate your policy using both the MC and TD(0) algorithms. In the cells below execute the required functions and display the results. For the `MC_state_values` function use 10,000 episodes.  For the `td_0_state_values` function use 20,000 episodes, a learning rate of 0.01 and a discount rate of 1.0. 

Compare the state values computed by the two algorithms. Do both methods compute state-values consistent with the problem, and why? Why do you think these estimates of state value are different? 

ANS 1:   

ANS 2:   

## Apply Double Q-Learning

Finally, you will apply Double Q-learning(0) to the warehouse navigation problem. 

### Q-Learning Environment Simulator

Since Q-learning is different from MC or TD/SARSA learning, you will need to create a new set of environment simulator functions. Keep in mind that the simulator is not part of the agent and the agent can only interact with the environment through the simulations. 

You may use the Q-learning simulator code from the Q-learning notebook as a starting point to create the following functions. It will help to change the names of these functions, by say, prefixing Q_ to prevent name space conflicts with previously defined functions. 


**Q_action_lookup** and **Q_index_lookup** functions to translate between an index and the corresponding action or the action and the corresponding index.  

**Q_next_state** function returns the next state given arguments of state and action. 

**Q_simulate_environment** function is similar to the `simulate_environment` function you have been working with. The arguments are state and action. The function then returns the successor or state, the an array of rewards from the successor state, and a flag to indicate if the successor state is terminal. 

### Random Episode Start

In the cell below, create and test a function to find a random start for each episode. You can define the `Q_start_episode` function starting with the function in the MC RL lesson notebook. The function takes arguments of number of states and number of actions. The function uses the `is_terminal` and `Q_simulate_environment` functions you defined earlier. The function returns the state, initial action index and the initial reward. 
    

### Double Q-Learning    

It is now time to pull together the functions to implement the double Q-learning algorithm. There are two functions, which you can use the functions from the MC lesson notebook as a starting point.   

**update_double_Q** is a function that computes the error, using the double Q update, and updates the Q value. The function takes the arguments of the two Q arrays, the current state, the action index, the current reward, the learning rate (alpha) and the discount rate (gamma). The function uses the `Q_simulate_environment` function. The function returns the updated Q array, the successor state, the successor reward, a flag indicating if the successor state is terminal and the index of the successor action.   

**double_Q_learning_0** is a function that randomly updates one or the other Q arrays using the Q array as an argument over the specified number of episodes. The function has arguments of policy, number of episodes, the learning rate (alpha), and the discount rate (gamma). The function makes calls to the `update_double_Q` function. An updated Q array is returned.   

Execute this code using the initial random policy, 10,000 episodes, a learning rate of 0.01 and a discount rate of 1.0.   

Make sure that the action values of the taboo states and the terminal state are  all 0.

With the action values computed using the action values, you will now find an improved policy. Make sure you create a copy of the initial random policy with `copy.deepcopy`. Execute the `update_policy` function using the copy of the initial random policy, the action value array you computed, and with $\epsilon = 0.01$.

Now, you will evaluate your policy using the MC algorithm. In the cells below execute the required function with 10,000 episodes and display the results. 

Are these state values reasonably similar to those found with the SARSA and is this what you expect and why? 

ANS 1:   